In [1]:
#! pip install --upgrade git+https://github.com/sblack-usu/dataretrieval
from dataretrieval import nwis

The dataRetrieval package was created to simplify the process of loading hydrologic data into the R environment. It is designed to retrieve the major data types of U.S. Geological Survey (USGS) hydrologic data that are available on the Web, as well as data from the Water Quality Portal (WQP), which currently houses water quality data from the Environmental Protection Agency (EPA), U.S. Department of Agriculture (USDA), and USGS. Direct USGS data is obtained from a service called the National Water Information System (NWIS).

For information on getting started in R and installing the package, see Getting Started. Any use of trade, firm, or product names is for descriptive purposes only and does not imply endorsement by the U.S. Government.

A quick workflow for USGS dataRetrieval functions:

In [2]:
'''
library(dataRetrieval)
# Choptank River near Greensboro, MD
siteNumber <- "01491000" 
ChoptankInfo <- readNWISsite(siteNumber)
parameterCd <- "00060"

#Raw daily data:
rawDailyData <- readNWISdv(siteNumber,parameterCd,
                      "1980-01-01","2010-01-01")

# Sample data Nitrate:
parameterCd <- "00618"
qwData <- readNWISqw(siteNumber,parameterCd,
                      "1980-01-01","2010-01-01")

pCode <- readNWISpCode(parameterCd)
'''

# Choptank River near Greensboro, MD
siteNumber = '01491000'
chop_tank_info = nwis.get_record(sites=siteNumber, service='site')
parameterCd = '00060'

# raw daily data
rawDailyData = nwis.get_record(sites=siteNumber, service='dv', parameterCd=parameterCd, start="1980-01-01", end="2010-01-01")

# sample data Nitrate:
parameterCd = "00618"
qwData = nwis.get_record(sites=siteNumber, service='qwdata', parameterCd=parameterCd, start="1980-01-01", end="2010-01-01")

pCode = nwis.get_record(service='pmcodes', parameterCd=parameterCd)

TODO - table of dataretrieval functions

# USGS Web Retrievals TODO link up or remove links

In this section, examples of National Water Information System (NWIS) retrievals show how to get raw data into R. This data includes [site information](#site-information), measured [parameter information](#parameter-information), historical [daily values](#daily-data), [unit values](#unit-data) (which include real-time data but can also include other sensor data stored at regular time intervals), [water quality data](#water-quality-data), [groundwater level data](#groundwater-level-data), [peak flow data](#peak-flow-data), [rating curve data](#rating-curve-data),  [surface-water measurement data](#surface-water-measurement-data), [water use data](#water-use-data), and [statistics data](#statistics-data). The section [Embedded Metadata](#embedded-metadata) shows instructions for getting metadata that is attached to each returned data frame.

The USGS organizes hydrologic data in a standard structure.  Streamgages are located throughout the United States, and each streamgage has a unique ID (referred in this document and throughout the `dataRetrieval` package as `siteNumber`).  Often (but not always), these ID's are 8 digits for surface-water sites and 15 digits for groundwater sites.  The first step to finding data is discovering this `siteNumber`. There are many ways to do this, one is the [National Water Information System: Mapper](https://maps.waterdata.usgs.gov/mapper/index.html).

Once the `siteNumber` is known, the next required input for USGS data retrievals is the "parameter code".  This is a 5-digit code that specifies the measured parameter being requested.  For example, parameter code 00631 represents "Nitrate plus nitrite, water, filtered, milligrams per liter as nitrogen", with units of "mg/l as N". 

Not every station will measure all parameters. A short list of commonly measured parameters is shown in Table 2.

### TODO table here

Two output columns that may not be obvious are "srsname" and "casrn". Srsname stands for "Substance Registry Services". More information on the srs name can be found [here](http://ofmpub.epa.gov/sor_internet/registry/substreg/home/overview/home.do).

Casrn stands for "Chemical Abstracts Service (CAS) Registry Number". More information on CAS can be found [here](http://www.cas.org/content/chemical-substances/faqs).

For unit values data (sensor data measured at regular time intervals such as 15 minutes or hourly), knowing the parameter code and `siteNumber` is enough to make a request for data.  For most variables that are measured on a continuous basis, the USGS also stores the historical data as daily values.  These daily values are statistical summaries of the continuous data, e.g. maximum, minimum, mean, or median. The different statistics are specified by a 5-digit statistics code.  

Some common codes are shown in Table 3.

### TODO table here

Examples for using these site numbers, parameter codes, and statistic codes will be presented in subsequent sections.

There are occasions where NWIS values are not reported as numbers, instead there might be text describing a certain event such as “Ice”. Any value that cannot be converted to a number will be reported as NA in this package (not including remark code columns), unless the user sets an argument convertType to FALSE. In that case, the data is returned as a data frame that is entirely character columns.

## Site Information


### readNWISsite

Use the `readNWISsite` function to obtain all of the information available for a particular USGS site (or sites) such as full station name, drainage area, latitude, and longitude. `readNWISsite` can also access information about multiple sites with a vector input.

In [3]:
'''{r getSite, echo=TRUE, eval=FALSE}
siteNumbers <- c("01491000","01645000") 
siteINFO <- readNWISsite(siteNumbers)
'''
siteNumbers = ["01491000","01645000"]
siteINFO = nwis.get_record(sites=siteNumbers)

Site information is obtained from:
[https://waterservices.usgs.gov/rest/Site-Test-Tool.html](https://waterservices.usgs.gov/rest/Site-Test-Tool.html)

Information on the returned data can be found with the `comment` function as described in the [Metadata](#embedded-metadata) section.


In [ ]:
'''comment(siteINFO)'''
#TODO implement Metadata

### whatNWISdata

To discover what data is available for a particular USGS site, including measured parameters, period of record, and number of samples (count), use the `whatNWISdata` function. It is possible to limit the retrieval information to a subset of services. The possible choices for services are: "dv" (daily values), "uv", or "iv" (unit values), "qw" (water-quality), "sv" (sites visits), "pk" (peak measurements), "gw" (groundwater levels), "ad" (sites included in USGS Annual Water Data Reports External Link), "aw" (sites monitored by the USGS Active Groundwater Level Network External Link), and "id" (historical instantaneous values).

In the following example, we limit the retrieved data to only daily data. The default for "service" is `all`, which returns all of the available data for that site. Likewise, there are arguments for parameter code (`parameterCd`) and statistic code (`statCd`) to filter the results. The default for both is to return all possible values (`all`). The returned `count_nu` for "uv" data is the count of days with returned data, not the actual count of returned values.


In [ ]:
'''
# Continuing from the previous example:
# This pulls out just the daily, mean data:
dailyDataAvailable <- whatNWISdata(siteNumbers,
                    service="dv", statCd="00003")
'''

dailyDataAvailable = nwis.get_record(sites=siteNumbers, service="dv", statCd="00003")

### TODO build table


## Parameter Information

To obtain all of the available information concerning a measured parameter (or multiple parameters), use the `readNWISpCode` function:


In [5]:
'''
# Using defaults:
parameterCd <- "00618" 
parameterINFO <- readNWISpCode(parameterCd)
'''

pCode = nwis.get_record(service='pmcodes', parameterCd="00618")

,parameter_cd,parameter_group_nm,casrn,srsname,parameter_units
0,618,Nutrient,14797-55-8,Nitrate,mg/l as N


## Daily Data

To obtain daily records of USGS data, use the `readNWISdv` function. The arguments for this function are `siteNumber`, `parameterCd`, `startDate`, `endDate`, and `statCd` (defaults to "00003").  If you want to use the default values, you do not need to list them in the function call. Daily data is pulled from [https://waterservices.usgs.gov/rest/DV-Test-Tool.html](https://waterservices.usgs.gov/rest/DV-Test-Tool.html).

The dates (start and end) must be in the format "YYYY-MM-DD" (note: the user must include the quotes).  Setting the start date to "" (no space) will prompt the program to ask for the earliest date, and setting the end date to "" (no space) will prompt for the latest available date.


In [10]:
'''
# Choptank River near Greensboro, MD:
siteNumber <- "01491000"
parameterCd <- "00060"  # Discharge
startDate <- "2009-10-01"  
endDate <- "2012-09-30" 

discharge <- readNWISdv(siteNumber, 
                    parameterCd, startDate, endDate)
'''
# Choptank River near Greensboro, MD:
siteNumber = "01491000"
parameterCd = "00060"  # Discharge
startDate = "2009-10-01"  
endDate = "2012-09-30" 

discharge = nwis.get_record(sites=siteNumber, service='dv',
                    parameterCd=parameterCd, start=startDate, end=endDate)

The column "datetime" in the returned data frame is automatically imported as a variable of class "Date" in R. Each requested parameter has a value and remark code column.  The names of these columns depend on the requested parameter and stat code combinations. USGS daily value qualification codes are often "A" (approved for publication) or "P" (provisional data subject to revision). 

Another example would be a request for mean and maximum daily temperature and discharge in early 2012:


In [ ]:
'''
siteNumber <- "01491000"
parameterCd <- c("00010","00060")  # Temperature and discharge
statCd <- c("00001","00003")  # Mean and maximum
startDate <- "2012-01-01"
endDate <- "2012-05-01"

temperatureAndFlow <- readNWISdv(siteNumber, parameterCd, 
        startDate, endDate, statCd=statCd)
'''
siteNumber = "01491000"
parameterCd = ["00010","00060"]  # Temperature and discharge
statCd = ["00001","00003"]  # Mean and maximum
startDate = "2012-01-01"
endDate = "2012-05-01"

temperatureAndFlow = nwis.get_record(sites=siteNumber, service='dv', parameterCd=parameterCd, 
        start=startDate, end=endDate, statCd=statCd)

The column names can be shortened and simplified using the `renameNWISColumns` function.  This is not necessary, but may streamline subsequent data analysis and presentation. Site information, daily statistic information, and measured parameter information is attached to the data frame as attributes. This is discussed further in the [metadata](#embedded-metadata) section.

In [ ]:
'''names(temperatureAndFlow)'''
# tODO metadata

In [ ]:
'''temperatureAndFlow <- renameNWISColumns(temperatureAndFlow)
names(temperatureAndFlow)'''
## todo metadata and rename

In [ ]:
'''
#Information about the data frame attributes:
names(attributes(temperatureAndFlow))
'''
# todo metadata attributes

In [ ]:
'''
statInfo <- attr(temperatureAndFlow, "statisticInfo")
variableInfo <- attr(temperatureAndFlow, "variableInfo")
siteInfo <- attr(temperatureAndFlow, "siteInfo")
'''
# todo attr

In [ ]:
'''
variableInfo <- attr(temperatureAndFlow, "variableInfo")
siteInfo <- attr(temperatureAndFlow, "siteInfo")

par(mar=c(5,5,5,5)) #sets the size of the plot window

plot(temperatureAndFlow$Date, temperatureAndFlow$Wtemp_Max,
  ylab=variableInfo$parameter_desc[1],xlab="" )
par(new=TRUE)
plot(temperatureAndFlow$Date, temperatureAndFlow$Flow,
  col="red",type="l",xaxt="n",yaxt="n",xlab="",ylab="",axes=FALSE
  )
axis(4,col="red",col.axis="red")
mtext(variableInfo$parameter_desc[2],side=4,line=3,col="red")
title(paste(siteInfo$station_nm,"2012"))
legend("topleft", variableInfo$param_units, 
       col=c("black","red"),lty=c(NA,1),pch=c(1,NA))
'''

## Unit Data

Any data collected at regular time intervals (such as 15-minute or hourly) are known as "unit values". Many of these are delivered on a real time basis and very recent data (even less than an hour old in many cases) are available through the function `readNWISuv`.  Some of these unit values are available for many years, and some are only available for a recent time period such as 120 days.  Here is an example of a retrieval of such data.  


In [12]:
'''
parameterCd <- "00060"  # Discharge
startDate <- "2012-05-12" 
endDate <- "2012-05-13" 
dischargeUnit <- readNWISuv(siteNumber, parameterCd, 
        startDate, endDate)
dischargeUnit <- renameNWISColumns(dischargeUnit)
'''
siteNumber = "01491000"
parameterCd = "00060"  # Discharge
startDate = "2012-05-12" 
endDate = "2012-05-13" 
dischargeUnit = nwis.get_record(sites=siteNumber, service='iv', parameterCd=parameterCd, 
        start=startDate, end=endDate)
# TODO rename
#dischargeUnit <- renameNWISColumns(dischargeUnit)

The retrieval produces a data frame that contains 96 rows (one for every 15 minute period in the day). They include all data collected from the startDate through the endDate (starting and ending with midnight locally-collected time). The dateTime column is converted to UTC (Coordinated Universal Time), so midnight EST will be 5 hours earlier in the dateTime column (the previous day, at 7pm).

To override the UTC timezone, specify a valid timezone in the tz argument. Default is "", which will keep the dateTime column in UTC. Other valid timezones are:
```
America/New_York
America/Chicago
America/Denver
America/Los_Angeles
America/Anchorage
America/Honolulu
America/Jamaica
America/Managua
America/Phoenix
America/Metlakatla
```

Data are retrieved from [https://waterservices.usgs.gov/rest/IV-Test-Tool.html](https://waterservices.usgs.gov/rest/IV-Test-Tool.html). There are occasions where NWIS values are not reported as numbers, instead a common example is "Ice".  Any value that cannot be converted to a number will be reported as NA in this package. Site information and measured parameter information is attached to the data frame as attributes. This is discussed further in [metadata](#embedded-metadata) section.


## Water Quality Data

To get USGS water quality data from water samples collected at the streamgage or other monitoring site (as distinct from unit values collected through some type of automatic monitor) we can use the function `readNWISqw`, with the input arguments: `siteNumber`, `parameterCd`, `startDate`, and `endDate`. Additionally, the argument `expanded` is a logical input that allows the user to choose between a simple return of datetimes/qualifier/values (`expanded=FALSE`), or a more complete and verbose output (`expanded=TRUE`). `expanded = TRUE` includes such columns as remark codes, value qualifying text, and detection level for each parameter code. There also includes an argument "reshape", that converts the expanded dataset to a "wide" format (each requested parameter code gets individual columns). The defaults are `expanded=TRUE`, and `reshape=FALSE`. 


In [6]:
'''
# Dissolved Nitrate parameter codes:
parameterCd <- c("00618","71851")
startDate <- "1985-10-01"
endDate <- "2012-09-30"
dfLong <- readNWISqw(siteNumber, parameterCd, 
      startDate, endDate)
# Or the wide return:
dfWide <- readNWISqw(siteNumber, parameterCd,
      startDate, endDate, reshape=TRUE)
'''
siteNumber = "01491000"
parameterCd = ["00618","71851"]
startDate = "1985-10-01"
endDate = "2012-09-30"
dfLong = nwis.get_record(sites=siteNumber, service='qwdata', parameterCd=parameterCd, 
      start=startDate, end=endDate)
# Or the wide return:
'''dfWide = readNWISqw(siteNumber, parameterCd,
      startDate, endDate, reshape=TRUE)
      # TODO reshape
'''

'dfWide = readNWISqw(siteNumber, parameterCd,\n      startDate, endDate, reshape=TRUE)\n      # TODO reshape\n'

Site information and measured parameter information is attached to the data frame as attributes. This is discussed further in the [metadata](#embedded-metadata) section. Additional metadata, such as information about the column names can be found by using the `comment` function, also described in the [metadata](#embedded-metadata) section.


In [ ]:
'''comment(dfLong)'''
# TODO metadata

## Groundwater Level Data

Groundwater level measurements can be obtained with the `readNWISgwl` function. Information on the returned data can be found with the `comment` function, and attached attributes as described in the [metadata](#embedded-metadata) section.


In [ ]:
'''
siteNumber <- "434400121275801"
groundWater <- readNWISgwl(siteNumber)
'''
siteNumber = "434400121275801"
groundWater = nwis.get_record(sites=siteNumber, service='gwlevels')

## Peak Flow Data

Peak flow data are instantaneous discharge or stage data that record the maximum values of these variables during a flood event.  They include the annual peak flood event but can also include records of other peaks that are lower than the annual maximum. Peak discharge measurements can be obtained with the `readNWISpeak` function. Information on the returned data can be found with the `comment` function and attached attributes as described in the [metadata](#embedded-metadata) section.


In [ ]:
'''
siteNumber <- '01594440'
peakData <- readNWISpeak(siteNumber)
'''
siteNumber = '01594440'
peakData = nwis.get_record(sites=siteNumber, service='peaks')

## Rating Curve Data

Rating curves are the calibration curves that are used to convert measurements of stage to discharge.  Because of changing hydrologic conditions these rating curves change over time. Information on the returned data can be found with the `comment` function and attached attributes as described in the [metadata](#embedded-metadata) section.

Rating curves can be obtained with the `readNWISrating` function.


In [2]:
'''
ratingData <- readNWISrating(siteNumber, "base")
attr(ratingData, "RATING")
'''
ratings  = nwis.get_record(service='ratings', site='01594440')
ratings


,INDEP,DEP,STOR
0,2.99,30.00,*
1,4.00,110.00,*
2,5.00,225.00,*
3,5.50,300.00,*
4,6.00,390.00,*
5,6.50,490.00,*
6,7.00,600.00,*
7,9.00,1175.00,*
8,13.00,4350.00,*
9,20.85,16497.75,*


## Surface-Water Measurement Data

These data are the discrete measurements of discharge that are made for the purpose of developing or revising the rating curve.  Information on the returned data can be found with the `comment` function and attached attributes as described in the [metadata](#embedded-metadata) section.

Surface-water measurement data can be obtained with the `readNWISmeas` function.

In [ ]:
'''surfaceData <- readNWISmeas(siteNumber)'''
siteNumber = '01594440'
nwis.get_record(sites=siteNumber, service='measurements')

## Water Use Data
Retrieves water use data from USGS Water Use Data for the Nation.  See [https://waterdata.usgs.gov/nwis/wu](https://waterdata.usgs.gov/nwis/wu) for more information.  All available use categories for the supplied arguments are retrieved. 


In [2]:
'''
allegheny <- readNWISuse(stateCd = "Pennsylvania",
                         countyCd = "Allegheny")
national <- readNWISuse(stateCd = NULL, 
                        countyCd = NULL, 
                        transform = TRUE)
'''
from dataretrieval import nwis
allegheny = nwis.get_record(service="water_use", state="PA", counties="003")
allegheny
#national = nwis.get_record(service="water_use")
#national

,state_cd,state_name,county_cd,county_nm,year,"Total Population total population of area, in thousands","Public Supply population served by groundwater, in thousands","Public Supply population served by surface water, in thousands","Public Supply total population served, in thousands","Public Supply self-supplied groundwater withdrawals, fresh, in Mgal/d",...,"Hydroelectric Power power generated by instream use, in gigawatt-hours","Hydroelectric Power power generated by offstream use, in gigawatt-hours","Hydroelectric Power total power generated, in gigawatt-hours",Hydroelectric Power number of instream facilities,Hydroelectric Power number of offstream facilities,Hydroelectric Power total number of facilities,"Wastewater Treatment returns by public wastewater facilities, in Mgal/d",Wastewater Treatment number of public wastewater facilities,Wastewater Treatment number of wastewater facilities,"Wastewater Treatment reclaimed wastewater released by wastewater facilities, in Mgal/d"
0,42,Pennsylvania,3,Allegheny County,1985,1388.300,99.900,1227.030,1326.930,16.47,...,0.00,-,-,-,-,-,266.21,85,-,-
1,42,Pennsylvania,3,Allegheny County,1990,1336.450,221.260,1079.340,1300.600,13.51,...,0.00,-,-,0,-,-,42.51,35,-,-
2,42,Pennsylvania,3,Allegheny County,1995,1309.820,192.110,1090.800,1282.910,14.19,...,0.00,0.00,0.00,0,0,0,225.27,21,-,-
3,42,Pennsylvania,3,Allegheny County,2000,1281.670,-,-,1281.670,13.48,...,-,-,-,-,-,-,-,-,-,-
4,42,Pennsylvania,3,Allegheny County,2005,1235.841,64.264,1171.577,1235.841,0.45,...,-,-,-,-,-,-,-,-,-,-
5,42,Pennsylvania,3,Allegheny County,2010,1223.348,14.480,1021.370,1035.850,0.87,...,-,-,-,-,-,-,-,-,-,-
6,42,Pennsylvania,3,Allegheny County,2015,1230.459,91.855,980.108,1071.963,8.82,...,-,-,-,-,-,-,-,-,-,-


## Statistics Data
Retrieves site statistics from the USGS Statistics Web Service beta.  


In [ ]:
'''
discharge_stats <- readNWISstat(siteNumbers=c("02319394"),
                   parameterCd=c("00060"),
                   statReportType="annual") 
'''
nwis.get_stats(sites='02319394', parameterCd="00060", statReportType='annual', statTypeCd='all')


# Water Quality Portal Web Retrievals

There are additional water quality data sets available from the [Water Quality Data Portal](https://www.waterqualitydata.us/).  These data sets can be housed in either the STORET database (data from EPA), NWIS database (data from USGS), STEWARDS database (data from USDA), and additional databases are slated to be included in the future.  Because only USGS uses parameter codes, a "characteristic name" must be supplied.  The `readWQPqw` function can take either a USGS parameter code, or a more general characteristic name in the parameterCd input argument. The Water Quality Data Portal includes data discovery tools and information on characteristic names. The following example retrieves specific conductance from a DNR site in Wisconsin. 


In [2]:
'''
specificCond <- readWQPqw('WIDNR_WQX-10032762',
                          'Specific conductance',
                          '2011-05-01','2011-09-30')
'''
from dataretrieval import wqp
results = wqp.get_results(siteid='WIDNR_WQX-10032762',
                          characteristicName = 'Specific conductance',
                          startDateLo='05-01-2011', startDateHi='09-30-2011')
results

https://www.waterqualitydata.us:443/data/Result/Search?siteid=WIDNR_WQX-10032762&characteristicName=Specific+conductance&startDateLo=05-01-2011&startDateHi=09-30-2011&zip=no&mimeType=csv


,OrganizationIdentifier,OrganizationFormalName,ActivityIdentifier,ActivityTypeCode,ActivityMediaName,ActivityMediaSubdivisionName,ActivityStartDate,ActivityStartTime/Time,ActivityStartTime/TimeZoneCode,ActivityEndDate,...,ResultAnalyticalMethod/MethodName,MethodDescriptionText,LaboratoryName,AnalysisStartDate,ResultLaboratoryCommentText,DetectionQuantitationLimitTypeName,DetectionQuantitationLimitMeasure/MeasureValue,DetectionQuantitationLimitMeasure/MeasureUnitCode,PreparationStartDate,ProviderName
0,WIDNR_WQX,Wisconsin Department of Natural Resources,WIDNR_WQX-45822640,Field Msr/Obs,Water,NaN,2011-05-09,12:20:00,CDT,2011-05-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STORET
1,WIDNR_WQX,Wisconsin Department of Natural Resources,WIDNR_WQX-49176537,Field Msr/Obs,Water,NaN,2011-08-08,13:55:00,CDT,2011-08-08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STORET
2,WIDNR_WQX,Wisconsin Department of Natural Resources,WIDNR_WQX-47619240,Field Msr/Obs,Water,NaN,2011-07-06,08:35:00,CDT,2011-07-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STORET
3,WIDNR_WQX,Wisconsin Department of Natural Resources,WIDNR_WQX-50689894,Field Msr/Obs,Water,NaN,2011-09-11,16:10:00,CDT,2011-09-11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STORET
4,WIDNR_WQX,Wisconsin Department of Natural Resources,WIDNR_WQX-46495059,Field Msr/Obs,Water,NaN,2011-06-05,14:45:00,CDT,2011-06-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STORET


A tool for finding NWIS characteristic names can be found [here](https://www.waterqualitydata.us/public_srsnames/)

# Generalized Retrievals

The previous examples all took specific input arguments: `siteNumber`, `parameterCd` (or characteristic name), `startDate`, `endDate`, etc. However, the Web services that supply the data can accept a wide variety of additional arguments. 

## NWIS 

### Sites: whatNWISsites

The function `whatNWISsites` can be used to discover NWIS sites based on any query that the NWIS Site Service offers. This is done by using the `...` argument, which allows the user to use any arbitrary input argument. We can then use the service [here](https://waterservices.usgs.gov/rest/Site-Test-Tool.html) to discover many options for searching for NWIS sites. For example, you may want to search for sites in a lat/lon bounding box, or only sites tidal streams, or sites with water quality samples, sites above a certain altitude, etc. The results of this site query generate a URL. For example, the tool provided a search within a specified bounding box, for sites that have daily discharge (parameter code = 00060) and temperature (parameter code = 00010). The generated URL is:

[https://waterservices.usgs.gov/nwis/site/?format=rdb&bBox=-83.0,36.5,-81.0,38.5&parameterCd=00010,00060&hasDataTypeCd=dv](https://waterservices.usgs.gov/nwis/site/?format=rdb&bBox=-83.0,36.5,-81.0,38.5&parameterCd=00010,00060&hasDataTypeCd=dv)

The following `dataRetrieval` code can be used to get those sites:

In [1]:
# TODO what sites not implemented
'''
sites <- whatNWISsites(bBox=c(-83.0,36.5,-81.0,38.5), 
                      parameterCd=c("00010","00060"),
                      hasDataTypeCd="dv")
'''
from dataretrieval import nwis
results = nwis.what_sites(bBox=[-83.0,36.5,-81.0,38.5],
                         parameterCd=["00010","00060"],
                         hasDataTypeCd="dv")
results

,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,alt_datum_cd,huc_cd
0,USGS,03161500,"SOUTH FORK NEW RIVER NR CRUMPLER, NC",ST,36.534847,-81.342877,S,NAD83,2550.00,1.00,NGVD29,5050001
1,USGS,03163000,"NEW RIVER NEAR BAYWOOD, VA",ST,36.608461,-81.044250,U,NAD83,2260.00,1.00,NGVD29,5050001
2,USGS,03175100,"COX BRANCH AB TAZEWELL RESERVOIR NR GRATTON, VA",ST,37.153449,-81.413995,U,NAD83,2990.00,1.00,NGVD29,5050002
3,USGS,03175140,"WEST FORK COVE CREEK NEAR BLUEFIELD, VA",ST,37.184283,-81.329824,S,NAD83,2650.00,20.00,NGVD29,5050002
4,USGS,03177700,"BLUESTONE RIVER AT BLUEFIELD, VA",ST,37.255950,-81.281766,U,NAD83,2350.00,1.00,NGVD29,5050002
5,USGS,03177710,"BLUESTONE RIVER AT FALLS MILLS, VA",ST,37.271506,-81.304823,U,NAD83,2316.68,0.60,NAVD88,5050002
6,USGS,03178000,"BLUESTONE R NR SPANISHBURG, WV",ST,37.433472,-81.110000,1,NAD83,2051.13,10.00,NGVD29,5050002
7,USGS,03178500,"CAMP CREEK NEAR CAMP CREEK, WV",ST,37.504284,-81.127595,U,NAD83,NaN,NaN,NaN,5050002
8,USGS,03179000,"BLUESTONE RIVER NEAR PIPESTEM, WV",ST,37.544008,-81.010368,U,NAD83,1526.91,10.00,NAVD88,5050002
9,USGS,03185000,"PINEY CREEK AT RALEIGH, WV",ST,37.760671,-81.162321,U,NAD83,2083.90,0.10,NAVD88,5050004


### Data: readNWISdata

For NWIS data, the function `readNWISdata` can be used. The argument listed in the R help file is `...` and `service` (only for data requests). Table 5 describes the services are available.

TODO table data

The `...` argument allows the user to create their own queries based on the instructions found in the web links above. The links provide instructions on how to create a URL to request data. Perhaps you want sites only in Wisconsin, with a drainage area less than 50 mi<sup>2</sup>, and the most recent daily discharge data. That request would be done as follows:


In [1]:
'''
dischargeWI <- readNWISdata(service="dv",
                           stateCd="WI",
                           parameterCd="00060",
                           drainAreaMin="50",
                           statCd="00003")

# TODO, implement attr metadata
siteInfo <- attr(dischargeWI, "siteInfo")
'''
dischargeWI = nwis.get_record(sites=None, service="dv",
                           stateCd="WI",
                           parameterCd="00060",
                           drainAreaMin="50",
                           statCd="00003")

NameError: name 'nwis' is not defined

## WQP 

TODO only have implementations 
Just as with NWIS, the Water Quality Portal (WQP) offers a variety of ways to search for sites and request data. The possible Web service arguments for WQP site searches is found [here](https://www.waterqualitydata.us/webservices_documentation).

### Sites: whatWQPsites

To discover available sites in the WQP in New Jersey that have measured Chloride, use the function `whatWQPsites`.


In [3]:
'''
sitesNJ <- whatWQPsites(statecode="US:34",
                       characteristicName="Chloride")
'''
from dataretrieval import wqp
sitesNJ = wqp.what_sites(statecode="US:34", characteristicName="Chloride")

### Data: readWQPdata

To get data from the WQP using generalized Web service calls, use the function `readWQPdata`. For example, to get all the pH data in Wisconsin:


In [ ]:
'''
dataPH <- readWQPdata(statecode="US:55", 
                 characteristicName="pH")
'''
dataPH = wqp.what_sites(statecode="US:55", characteristicName="pH")

### Availability: whatWQPdata

The function `whatWQPdata` returns a data frame with information on the amount of data collected at a site. For example:


In [ ]:
'''
type <- "Stream"
sites <- whatWQPdata(countycode="US:55:025",siteType=type)
'''
# TODO this doesn't work an error handling in wqp implementation sucks
streamType = "Stream"
sites = wqp.get_results(countycode="US:55:025", siteType=streamType)

This returns a data frame with all of the sites that were measured in streams in Dane County, WI. Also, in that table, there is a measure of `activityCount` (how often the site was sampled), and `resultCount` (how many individual results are available).


### Samples: whatWQPsamples

The function `whatWQPsamples` returns information on the individual samples collected at a site. For example:


In [ ]:
'''site <- whatWQPsamples(siteid="USGS-01594440")'''

site = wqp.what_sites(siteid="USGS-01594440")

This returns one row for each instance that a sample was collect.

### Metrics: whatWQPmetrics

The function `whatWQPmetrics` provides metric information. This is only currently available for STORET data:


In [ ]:
'''
type <- "Stream"
sites <- whatWQPmetrics(countycode="US:55:025",siteType=type)
'''
streamType = "Stream"
wqp.what_sites(countycode="US:55:025",siteType=streamType)

# Embedded Metadata

TODO not implemented at all from what I can tell

All data frames returned from the Web services have some form of associated metadata. This information is included as attributes to the data frame. All data frames will have a `url` (returning a character of the url used to obtain the data), `siteInfo` (returning a data frame with information on sites),  and `queryTime` (returning a POSIXct datetime) attributes. For example, the url and query time used to obtain the data can be found as follows:

```{r meta1, eval=FALSE}
attr(dischargeWI, "url")
attr(dischargeWI, "queryTime")
siteInfo <- attr(dischargeWI, "siteInfo")
```

Depending on the format that the data was obtained (RDB, WaterML1, etc), there will be additional information embedded in the data frame as attributes. To discover the available attributes:

```{r meta2, eval=FALSE}
names(attributes(dischargeWI))
```

For data obtained from `readNWISuv`, `readNWISdv`, `readNWISgwl` there are two attributes that are particularly useful: `siteInfo` and `variableInfo`.

```{r meta3, eval=FALSE}
siteInfo <- attr(dischargeWI, "siteInfo")
variableInfo <- attr(dischargeWI, "variableInfo")
```

Data obtained from `readNWISpeak`, `readNWISmeas`, and `readNWISrating`, the `comment` attribute is useful.

```{r meta5, eval=FALSE}
comment(peakData)
#Which is equivalent to:
attr(peakData, "comment")
```